Use this notebook to generate TTS and listen to the result in this notebook.

Use `tts_model` to change to a different ElevenLabs model and use `tts_voice` to change to a different voice.

You can use an `input_text` to supply a string directly in the notebook, or use a `input_file` to supply a path to a text file (e.g. `input.txt`).

You can also optionally an `output_file` to save the generated audio to a file (e.g. `output.mp3`).

Press `Run All` and the TTS audio will render at the bottom after a while.

In [ ]:
import os
from src.ai import elevenlabs
from src.utils import saveB64Audio, displayAudio, chunk_text, get_text_from_url
from src.enums import ElevenLabsTTSModel, ElevenLabsTTSVoice
from src.format_tts import format_tts_text

# change the model and/or voice if desired
# TODO support openai tts
tts_model = ElevenLabsTTSModel.Multilingual_v2.value
tts_voice = ElevenLabsTTSVoice.Brian.value # TODO add more voices to enum

# choose 1 of the below inputs
input_text = ""
input_file = ''
input_url = ''
reformat_url_text = True

# optionally save the audio
output_file = ''

In [ ]:
if not input_text and not os.path.exists(input_file) and not input_url:
	raise SystemExit('Please provide a valid input text, file or url')

if input_url:
	input_text = get_text_from_url(input_url)
	if reformat_url_text:
		res = format_tts_text(input_text)
		input_text = res
		print('Reformatted text:')
		print(res)

if input_file and os.path.exists(input_file):
	with open(input_file, 'r') as f:
		input_text = f.read()

if not input_text:
	raise SystemExit('Could not find any input text')

chunks = []
if len(input_text) > 5000:
	chunks = chunk_text(input_text, 5000)
else:
	chunks.append(input_text)
has_multiple_chunks = len(chunks) > 1

In [ ]:
for i, chunk in enumerate(chunks):
	audio = elevenlabs.getSpeechB64(chunk, tts_model, tts_voice)
	if output_file:
		if has_multiple_chunks:
			chunk_output_file = f"{os.path.splitext(output_file)[0]}_part{i+1}{os.path.splitext(output_file)[1]}"
		else:
			chunk_output_file = output_file
		saveB64Audio(audio, chunk_output_file)
		print(f'Audio saved to {chunk_output_file}')
	displayAudio(audio)